## GOES Irradiance Corrections
To clip each band appropriately, we need to convert raw radiance values to reflectivity (channels 2 and 5) and brightness temperature (channel 13). Loosely based off of Brian Blaylock's goes2go

In [6]:
import goes2go as g2g
import xarray as xr

In [12]:
def combine_goes_channels(date):
    date = str(date)
    path = '/storage/cdalden/goes/goes16/'
    ds_C02 = xr.open_dataset(path + f'channel02/goes16_C02_colorado_{date}.zarr')
    ds_C05 = xr.open_dataset(path + f'channel05/goes16_C05_colorado_{date}.zarr')
    ds_C13 = xr.open_dataset(path + f'channel13/goes16_C13_colorado_{date}.zarr')

    ds_C13['red'] = g2g.rgb.normalize(ds_C13['Rad'], -53.5, 7.5, clip=True)
    ds_C02['green'] = g2g.rgb.normalize(ds_C02['Rad'], 0, 0.78, clip=True)
    ds_C05['blue'] = g2g.rgb.normalize(ds_C05['Rad'], 0.01, 0.59, clip=True)

    # Combine the normalized variables into a new dataset
    ds_RGB = xr.Dataset({
        'red': ds_C13['red'],
        'green': ds_C02['green'],
        'blue': ds_C05['blue']
    })

    return ds_RGB


In [13]:
combine_goes_channels('20230604')

/tmp/ipykernel_17800/2535091711.py:6: UserWarning: The specified chunks separate the stored chunks along dimension "y" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  ds_C02 = xr.open_dataset(path + f'channel02/goes16_C02_colorado_{date}.zarr',
/tmp/ipykernel_17800/2535091711.py:6: UserWarning: The specified chunks separate the stored chunks along dimension "x" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  ds_C02 = xr.open_dataset(path + f'channel02/goes16_C02_colorado_{date}.zarr',
/tmp/ipykernel_17800/2535091711.py:8: UserWarning: The specified chunks separate the stored chunks along dimension "y" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  ds_C05 = xr.open_dataset(path + f'channel05/goes16_C05_colorado_{date}.zarr',
/tmp/ipykernel_17800/2535091711.py:8: UserWarning: The specified chunks separate the stored chunks along d

MemoryError: Unable to allocate 1.52 GiB for an array with shape (864, 1120, 1689) and data type bool

In [14]:
date = str('20230604')
path = '/storage/cdalden/goes/goes16/'
ds_C13 = xr.open_dataset(path + f'channel13/goes16_C13_colorado_{date}.zarr')

ds_C13

<xarray.Dataset> Size: 133MB
Dimensions:                                           (t: 288, y: 160,
                                                       num_star_looks: 24,
                                                       band: 1,
                                                       number_of_harmonization_coefficients: 3,
                                                       number_of_image_bounds: 2,
                                                       x: 241,
                                                       number_of_time_bounds: 2)
Coordinates:
    x_image                                           float32 4B ...
  * y                                                 (y) float64 1kB 0.1092 ...
    t_star_look                                       (num_star_looks) datetime64[ns] 192B ...
    band_wavelength                                   (band) float32 4B ...
    y_image                                           float32 4B ...
    band_id                                           (band) int8 1B ...
  * x                                                 (x) float64 2kB -0.0746...
  * t                                                 (t) datetime64[ns] 2kB ...
    band_wavelength_star_look                         (num_star_looks) float32 96B ...
Dimensions without coordinates: num_star_looks, band,
                                number_of_harmonization_coefficients,
                                number_of_image_bounds, number_of_time_bounds
Data variables: (12/39)
    algorithm_dynamic_input_data_container            (t) int32 1kB ...
    focal_plane_temperature_threshold_exceeded_count  (t) float64 2kB ...
    nominal_satellite_height                          (t) float32 1kB ...
    planck_bc2                                        (t) float32 1kB ...
    algorithm_product_version_container               (t) int32 1kB ...
    processing_parm_version_container                 (t) int32 1kB ...
    ...                                                ...
    saturated_pixel_count                             (t) float64 2kB ...
    nominal_satellite_subpoint_lon                    (t) float32 1kB ...
    goes_imager_projection                            (t) int32 1kB ...
    y_image_bounds                                    (t, number_of_image_bounds) float32 2kB ...
    a_h_NRTH                                          (t, number_of_harmonization_coefficients) float32 3kB ...
    planck_fk1                                        (t) float32 1kB ...
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    production_site:           RBU
    ...                        ...
    timeline_id:               ABI Mode 6
    date_created:              2023-06-04T00:04:03.6Z
    time_coverage_start:       2023-06-04T00:01:17.7Z
    time_coverage_end:         2023-06-04T00:03:56.3Z
    LUT_Filenames:             SpaceLookParams(FM1A_ADR1270_PR_12_02_01)-7371...
    id:                        4c559449-9e4c-4311-af40-e8ceeb6376f5